In [ ]:
import pandas as pd


In [ ]:
fft = pd.read_csv('Transfos_data.csv', sep = ";")
fft.head()

In [ ]:
fft2_men = fft[fft['ORIGIN-'] == 'TRAIN'][fft['SEXE-']==1].drop(['SEXE-', 'ORIGIN-', 'ID'], axis = 1)
fft2_women = fft[fft['ORIGIN-'] == 'TRAIN'][fft['SEXE-']==0].drop(['SEXE-', 'ORIGIN-', 'ID'], axis = 1)

In [ ]:
import numpy as np

def cdf(sample, x, sort = False):
    # Sorts the sample, if unsorted
    if sort:
        sample.sort()
    # Counts how many observations are below x
    cdf = sum(sample <= x)
    # Divides by the total number of observations
    cdf = cdf / len(sample)
    return cdf

def ks_2samp(sample1, sample2):
    # Gets all observations
    observations = np.concatenate((sample1, sample2))
    observations.sort()
    # Sorts the samples
    sample1.sort()
    sample2.sort()
    # Evaluates the KS statistic
    D_ks = [] # KS Statistic list
    for x in observations:
        cdf_sample1 = cdf(sample = sample1, x  = x)
        cdf_sample2 = cdf(sample = sample2, x  = x)
        D_ks.append(abs(cdf_sample1 - cdf_sample2))
    ks_stat = max(D_ks)
    # Calculates the P-Value based on the two-sided test
    # The P-Value comes from the KS Distribution Survival Function (SF = 1-CDF)
    m, n = float(len(sample1)), float(len(sample2))
    en = m * n / (m + n)
    p_value = stats.kstwo.sf(ks_stat, np.round(en))
    return {"ks_stat": ks_stat, "p_value" : p_value}

In [ ]:
import scipy.stats as stats
col_select = []
for col in fft2_men.columns:
    y = ks_2samp(fft2_men[col].to_numpy(),fft2_women[col].to_numpy())
    # On ne rejette pas l'hypothèse de distribution 
    if y['p_value']> 0.1:
        col_select.append(col)

